# Importation

In [ ]:
!pip install datasets

In [2]:
from torchvision import models, transforms
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer
from torch.utils.data import Subset
from torchvision import transforms
from datasets import load_dataset
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import argparse
import random
import pickle
import torch
import mmap

In [3]:
device = torch.device("cuda")

print(device)

cuda


In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Hyperparamters

In [42]:
batch_size = 64
block_size = 256
max_iters = 3000
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

# Data Processing

In [34]:
print("Loading dataset...")
dataset = load_dataset("glaiveai/glaive-code-assistant")
print("Dataset loaded.")

Loading dataset...
Dataset loaded.


In [35]:
train_test_split = dataset['train'].train_test_split(test_size=0.2)

train_data = train_test_split['train']
val_data = train_test_split['test']

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

Training set size: 108887
Validation set size: 27222


In [36]:
train_dataset = train_data
val_dataset = val_data

print("Train sample:", train_dataset[0])
print("Validation sample:", val_dataset[0])

Train sample: {'question': 'I have a list of numbers and I want to calculate the sum of all the numbers. Can someone provide me with a code example using C programming language?', 'answer': 'Certainly! Here\'s a code example in C that calculates the sum of a given list of numbers:\n\n```c\n#include<stdio.h>\n\nint main()\n{\n    int array[] = {1, 10, 100};\n    int size = 3;\n    int sum = 0;\n    \n    for (int i = 0; i < size; i++) {\n        sum = sum + array[i];\n    }\n    \n    printf("Sum of the array numbers is %d", sum);\n\n    return 0;\n}\n```\n\nIn this code, we first declare an array called `array` with three elements: 1, 10, and 100. The variable `size` is set to 3 to represent the number of elements in the array.\n\nWe then initialize a variable `sum` to 0, which will store the sum of the numbers.\n\nNext, we use a `for` loop to iterate through each element of the array. The loop starts with `i` as 0 and continues until `i` is less than `size`. In each iteration, we add 

In [37]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]'})

def convert_to_tensor_batch(dataset, batch_size=32):
    input_ids = []
    attention_masks = []

    for i in range(0, len(dataset), batch_size):
        batch = [dataset[j] for j in range(i, min(i + batch_size, len(dataset)))]

        texts = [entry['question'] + " [SEP] " + entry['answer'] for entry in batch]

        encoding = tokenizer(texts,
                             return_tensors="pt",
                             padding='max_length',
                             truncation=True,
                             max_length=block_size,
                             add_special_tokens=True)

        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

train_input_ids, train_attention_masks = convert_to_tensor_batch(train_dataset)
val_input_ids, val_attention_masks = convert_to_tensor_batch(val_dataset)

print(f"Train input_ids shape: {train_input_ids.shape}")
print(f"Train attention_masks shape: {train_attention_masks.shape}")
print(f"Validation input_ids shape: {val_input_ids.shape}")
print(f"Validation attention_masks shape: {val_attention_masks.shape}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Train input_ids shape: torch.Size([108887, 256])
Train attention_masks shape: torch.Size([108887, 256])
Validation input_ids shape: torch.Size([27222, 256])
Validation attention_masks shape: torch.Size([27222, 256])


In [38]:
def get_random_chunk(split):
    dataset = train_input_ids if split == 'train' else val_input_ids

    if len(dataset) < batch_size:
        print(f"Dataset too small for batch size ({batch_size}). Returning full dataset.")
        return dataset

    start_pos = random.randint(0, len(dataset) - batch_size)

    batch_samples = dataset[start_pos:start_pos + batch_size]

    return batch_samples

def get_batch(split):
    data = get_random_chunk(split)

    x = data[:, :-1]
    y = data[:, 1:]

    x, y = x.to(device), y.to(device)

    return x, y

x_train, y_train = get_batch('train')
x_val, y_val = get_batch('val')

if x_train is not None and y_train is not None:
    print(f"Train batch x shape: {x_train.shape}")
    print(f"Train batch y shape: {y_train.shape}")
if x_val is not None and y_val is not None:
    print(f"Validation batch x shape: {x_val.shape}")
    print(f"Validation batch y shape: {y_val.shape}")

Train batch x shape: torch.Size([64, 255])
Train batch y shape: torch.Size([64, 255])
Validation batch x shape: torch.Size([64, 255])
Validation batch y shape: torch.Size([64, 255])


# Model Architecture

In [22]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)


        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def resize_token_embeddings(self, new_vocab_size):
        old_embedding = self.token_embedding_table
        self.token_embedding_table = nn.Embedding(new_vocab_size, old_embedding.embedding_dim)
        self.token_embedding_table.weight.data[:old_embedding.num_embeddings] = old_embedding.weight.data

        old_lm_head = self.lm_head
        self.lm_head = nn.Linear(old_lm_head.in_features, new_vocab_size)
        self.lm_head.weight.data[:old_lm_head.out_features] = old_lm_head.weight.data

    def forward(self, index, targets=None):
        B, T = index.shape

        # print(f"index shape: {index.shape}")
        # if targets is not None:
        #    print(f"targets shape: {targets.shape}")

        # print(f"Max token ID: {index.max().item()}, Min token ID: {index.min().item()}")
        assert index.max().item() < vocab_size, f"Token ID exceeds vocab_size ({vocab_size})"
        assert index.min().item() >= 0, "Token ID is negative"

        assert T <= block_size, f"Sequence length T ({T}) exceeds block_size ({block_size})"

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(min(T, block_size), device=device))

        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss




    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

vocab_size = tokenizer.vocab_size
model = GPTLanguageModel(vocab_size)
m = model.to(device)

# Training Loop

In [39]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [40]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'sep_token': '[SEP]'})

vocab_size = len(tokenizer)
print(f"Updated vocab size: {vocab_size}")

model = GPTLanguageModel(vocab_size)

model.resize_token_embeddings(vocab_size)

model = model.to(device)


Updated vocab size: 50259


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"Final training loss: {loss.item()}")

torch.save(model.state_dict(), 'model-01.pt')
print('Model saved as model-01.pt')


# Example Usage


In [43]:
model = GPTLanguageModel(vocab_size)
model.load_state_dict(torch.load('model-01.pt'))
model = model.to(device)
model.eval()

prompt_question = "How do I create a Python function to calculate factorial?"

inputs = tokenizer(prompt_question, return_tensors='pt', truncation=True, padding=True, max_length=block_size).to(device)

with torch.no_grad():
    generated_ids = model.generate(inputs['input_ids'], max_new_tokens=200)

generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated code:", generated_code)


<ipython-input-43-525c772ff754>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model-01.pt'))


Generated code: How do I create a Python function to calculate factorial?  You can create a Python function named "factorial" to calculate factorial of a number. Here's an example:

```python
def factorial(n):
    # Def loop through the string
    print(n) # Loop until the given number is reached
    # Calculate factorial of 1
    for i in range(n+1, n+1):
        for j in range(i+1, n+1):
           print(i-1)
    # Print the factorial of sys
    return n * factorial(i+1)
```

In the provided code example, the function is `factorial` that takes an integer `n` as a parameter. It loops through the range from 2 to `n+1`. Inside the loop
